# ARTIST ANALYSIS: SUCCESS RATES, TIMELINES, METRICS

In [29]:
import pandas as pd
df_artists = pd.read_csv("artists_df.csv")
df_timeline = pd.read_csv("timeline_df.csv")
df_artists = df_artists.set_index("Unnamed: 0").rename_axis("name", axis = 0)
df_timeline = df_timeline.set_index("Unnamed: 0").rename_axis("release_date", axis = 0)

In [30]:
(pd.DataFrame(df_artists.corr())[pd.DataFrame(abs(df_artists.corr())) > 0.5].notnull()*1).sum().sum() - len(pd.DataFrame(df_artists.corr()).index)

10

In [31]:
corr_matrix = pd.DataFrame(df_artists.corr().fillna(0))
corr_pos = corr_matrix[corr_matrix > 0.5].notnull()
corr_neg = corr_matrix[corr_matrix < -0.5].notnull()
columns = indexes = corr_pos.columns
for row in range(len(indexes)):
    for col in range(len(columns)):
        if(row <= col):
            pass
        elif(corr_pos.iloc[row, col] == True):
            print(indexes[row], " <-> ", columns[col], "(", corr_matrix.iloc[row, col], ")")
        elif(corr_neg.iloc[row, col] == True):
            print(indexes[row], " <-> ", columns[col], "(", corr_matrix.iloc[row, col], ")")

acousticness  <->  energy ( -0.7607925963953158 )
valence  <->  danceability ( 0.582577277272758 )
explicit  <->  speechiness ( 0.7454039670173142 )
duration_min  <->  liveness ( 0.664299035047065 )
track_count  <->  collab ( -0.6043900792446417 )


In [33]:
dfx = df_artists[(1-df_artists["collab"])*df_artists["track_count"] > 5]
dfx

,danceability,energy,key,mode,speechiness,acousticness,instrumentalness,liveness,valence,popularity,explicit,available_markets,tempo,loudness,duration_min,collab,track_count
name,,,,,,,,,,,,,,,,,
Eminem,0.800684,0.785737,5.631579,0.578947,0.220463,0.155246,0.000045,0.272805,0.552600,0.784211,1.0,0.986676,3.526316,3.947368,4.578947,0.684211,19
The Beatles,0.513100,0.495600,6.400000,0.800000,0.035740,0.400330,0.025052,0.118130,0.521100,0.770000,0.0,1.000000,2.900000,3.500000,3.100000,0.500000,20
AC/DC,0.518700,0.850500,5.700000,0.700000,0.101790,0.057967,0.018540,0.208410,0.515100,0.761000,0.0,1.000000,2.300000,2.000000,4.300000,0.500000,20
Elton John,0.597579,0.593105,5.105263,0.789474,0.047653,0.311295,0.013135,0.140721,0.578000,0.750526,0.0,0.981346,3.421053,4.000000,4.578947,0.526316,19
Bob Marley & The Wailers,0.796800,0.502000,6.800000,0.800000,0.181440,0.171593,0.002398,0.156200,0.708200,0.744000,0.0,1.000000,3.200000,3.700000,4.200000,0.500000,20
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
The Shirelles,0.555400,0.540500,5.800000,0.800000,0.045190,0.556900,0.000161,0.271470,0.673800,0.414000,0.0,0.950633,3.700000,4.400000,2.500000,0.500000,20
The Miracles,0.529000,0.617444,4.722222,0.722222,0.038983,0.571411,0.002932,0.194883,0.645000,0.409444,0.0,1.000000,2.888889,3.777778,3.555556,0.555556,18
Carl Perkins,0.593000,0.735895,6.894737,0.789474,0.199647,0.494842,0.003046,0.406305,0.851316,0.375263,0.0,0.994670,2.631579,4.210526,2.526316,0.526316,19


In [24]:
(pd.DataFrame(dfx.corr())[pd.DataFrame(abs(dfx.corr())) > 0.5].notnull()*1).sum().sum() - len(pd.DataFrame(dfx.corr()).index)

16

In [27]:
dfx.corr()[pd.DataFrame(abs(dfx.corr())) > 0.5].notnull()*1

,danceability,energy,key,mode,speechiness,acousticness,instrumentalness,liveness,valence,popularity,explicit,available_markets,tempo,loudness,duration_min,collab,track_count
danceability,1,1,0,0,1,1,0,0,0,0,0,0,0,0,0,0,0
energy,1,1,0,0,0,1,0,0,1,0,0,0,0,0,0,0,0
key,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0
mode,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0
speechiness,1,0,0,0,1,0,0,0,0,0,1,0,0,0,0,0,0
acousticness,1,1,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0
instrumentalness,0,0,0,0,0,0,1,0,0,0,0,0,0,0,1,0,0
liveness,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0
valence,0,1,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0
popularity,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0


In [28]:
corr_matrix = pd.DataFrame(dfx.corr().fillna(0))
corr_pos = corr_matrix[corr_matrix > 0.5].notnull()
corr_neg = corr_matrix[corr_matrix < -0.5].notnull()
columns = indexes = corr_pos.columns
for row in range(len(indexes)):
    for col in range(len(columns)):
        if(row <= col):
            pass
        elif(corr_pos.iloc[row, col] == True):
            print(indexes[row], " <-> ", columns[col], "(", corr_matrix.iloc[row, col], ")")
        elif(corr_neg.iloc[row, col] == True):
            print(indexes[row], " <-> ", columns[col], "(", corr_matrix.iloc[row, col], ")")

energy  <->  danceability ( 0.5349686307877004 )
speechiness  <->  danceability ( 0.5316724120403105 )
acousticness  <->  danceability ( -0.6479599360212238 )
acousticness  <->  energy ( -0.8392449562620778 )
valence  <->  energy ( 0.6019641829429561 )
explicit  <->  speechiness ( 0.7128207528751032 )
duration_min  <->  instrumentalness ( 0.525985489174006 )
track_count  <->  collab ( -0.5878792178665332 )
